In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [3]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.39.1
Uninstalling transformers-4.39.1:
  Successfully uninstalled transformers-4.39.1
Found existing installation: accelerate 0.28.0
Uninstalling accelerate-0.28.0:
  Successfully uninstalled accelerate-0.28.0
  Using cached transformers-4.39.1-py3-none-any.whl.metadata (134 kB)
  Using cached accelerate-0.28.0-py3-none-any.whl.metadata (18 kB)
Using cached transformers-4.39.1-py3-none-any.whl (8.8 MB)
Using cached accelerate-0.28.0-py3-none-any.whl (290 kB)


In [4]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset, load_metric

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")

c:\Users\priks\anaconda3\envs\text-summarizer\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\priks\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [6]:
# Import necessary modules from the Hugging Face Transformers library
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Define the checkpoint or model identifier for the BART model
model_ckpt = "facebook/bart-large-cnn"

# Initialize a tokenizer using the pretrained BART model's tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

# Load the pretrained BART model for sequence-to-sequence tasks
# and move it to a specified device (e.g., GPU if available)
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)


In [7]:
# load the dataset, using samsum dataset from hugging face

from datasets import load_dataset

dataset_name = "samsum"
dataset = load_dataset(dataset_name)

In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [10]:
# Split the dataset into training, testing, and validation sets
train_dataset = dataset['train']
test_dataset = dataset['test']
validation_dataset = dataset['validation']

# Print information about the datasets
print("Training dataset:")
print(train_dataset)
print("Testing dataset:")
print(test_dataset)
print("Validation dataset:")
print(validation_dataset)

Training dataset:
Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 14732
})
Testing dataset:
Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 819
})
Validation dataset:
Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 818
})


In [17]:
print(f"text: \n{dataset['train'][1]['dialogue']}, \n\nsummary: \n{dataset['train'][1]['summary']}")

text: 
Olivia: Who are you voting for in this election? 
Oliver: Liberals as always.
Olivia: Me too!!
Oliver: Great, 

summary: 
Olivia and Olivier are voting for liberals in this election. 


In [ ]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['summary'], max_length = 128, truncation = True )

    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }
